In [2]:
from deeplake.core.vectorstore.deeplake_vectorstore import VectorStore
import torch
import time, os, shutil
from torchvision import transforms, models, datasets
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.models.feature_extraction import create_feature_extractor
from PIL import Image, ImageFilter

In [3]:
"""
Create ResNet34 model
(COULD BE PART OF A CLASS)
""" 
def resnet():
    # Check if GPU is available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Load the model
    model = models.resnet34(pretrained=True)

    # Create a feature extractor
    return_nodes = {
        'avgpool': 'embedding'
    }
    model = create_feature_extractor(model, return_nodes=return_nodes)

    # Freeze the model
    model.eval()
    model.to(device)
    return model

In [4]:
model = resnet()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def embedding_function(images, model=model, device=device, batch_size = 500):
    """Creates a list of embeddings based on a list of image filenames. Images are processed in batches."""
    transform= transforms.Compose([
        transforms.Resize((224,224)), 
        transforms.ToTensor(),
        transforms.Lambda(lambda x: torch.cat([x, x, x], dim=0) if x.shape[0] == 1 else x),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    if isinstance(images, str):
        images = [images]

    # Proceess the embeddings in batches, but return everything as a single list
    embeddings = []
    for i in range(0, len(images), batch_size):
        batch = torch.stack([transform(Image.open(item)) for item in images[i:i+batch_size]])
        batch = batch.to(device)
        with torch.no_grad():
            embeddings+= model(batch)['embedding'][:,:,0,0].cpu().numpy().tolist()

    return embeddings

/home/landon/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/landon/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
def create_all_vector_stores(vector_store_path="../data/vector_stores"):

    start_time = time.time()

    print("Loading Model")
    model = resnet()
    print("Model Loaded")

    # make_vector_store_dir(vector_store_path)

    print("Creating Vector Stores")
    # Path to the individual vector stores

    # Check if Vector Stores Folder exists and delete it if they do
    if os.path.exists(vector_store_path + "/subjectbox_vs"):
        shutil.rmtree(vector_store_path + "/subjectbox_vs")

    subjectbox_vs = VectorStore(
        path=f"{vector_store_path}/subjectbox_vs",
        tensor_params=[
            {"name": "image", "htype": "image", "sample_compression": "jpg"},
            {"name": "embedding", "htype": "embedding"},
            {"name": "filename", "htype": "text"},
        ],
    )
    print("Vector Stores Created")

    # Get the list of images
    sb_images = [f"../data/subjectbox/{item}" for item in os.listdir("../data/subjectbox/")]

    print("Adding Images to Vector Stores")

    # Get device
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    print("Device: ", device)

    # Add the images to the vector stores
    print("Subjectbox Images")
    subjectbox_vs.add(
        image=sb_images,
        filename=sb_images,
        embedding_function=embedding_function,
        embedding_data=sb_images,
    )

    print("Images Successfully Added to Vector Stores")

    end_time = time.time()

    print("Time to Create Vector Stores: ", round(end_time - start_time, 2), " seconds")
    avg_time = (end_time - start_time) / len(os.listdir('../data/subjectbox/'))
    print("Average Time per Image: ", round(avg_time, 4), " seconds")
    print("Time for all images: ", round(avg_time * len(os.listdir('../data/subjectbox/')) / 60, 2), " minutes")

    return None


# Test the function
create_all_vector_stores()

Loading Model


/home/landon/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/landon/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model Loaded
Creating Vector Stores


Vector Stores Created
Adding Images to Vector Stores
Device:  cuda
Subjectbox Images


Creating 19171 embeddings in 39 batches of size 500:: 100%|██████████| 39/39 [00:59<00:00,  1.53s/it]

Dataset(path='../data/vector_stores/subjectbox_vs', tensors=['image', 'embedding', 'filename', 'id'])

  tensor      htype               shape              dtype  compression
  -------    -------             -------            -------  ------- 
   image      image    (19171, 17:6079, 9:3880, 3)   uint8    jpeg   
 embedding  embedding         (19171, 512)          float32   None   
 filename     text             (19171, 1)             str     None   
    id        text             (19171, 1)             str     None   
Images Successfully Added to Vector Stores
Time to Create Vector Stores:  60.77  seconds
Average Time per Image:  0.0032  seconds
Time for all images:  1.01  minutes
